In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np

: 

: 

In [23]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 32 * 32)
        self.fc2 = nn.Linear(32 * 32, 32 * 32)
        self.fc3 = nn.Linear(32 * 32, 32 * 32)
        self.fc4 = nn.Linear(32 * 32, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        x = F.relu(x)
        output = F.log_softmax(x, dim=1)
        return output

In [24]:
# Define the training settings
batch_size = 64
test_batch_size = 1000
epochs = 10
lr = 1.0
gamma = 0.7
no_cuda = False
dry_run = False
seed = 1
log_interval = 10

In [25]:
use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")

In [26]:
train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1, 'pin_memory': True, 'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [27]:
dataset1 = datasets.MNIST('./mnist', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('./mnist', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/MNIST/raw

Processing...
Done!


In [28]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [29]:
for epoch in range(1, epochs + 1):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.298293
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.042123
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.875247
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.988765
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.578406
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.961531
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.418777
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.426981
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.471045
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.623374
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.656401
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.547445
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.533717
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.576160
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.256407
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.662797
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.487709
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.453242
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.299429
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.260876
Train Epoch: 1 [12800/60000 (

In [30]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

scheduler.step()

torch.save(model.state_dict(), "mnist.pth")


Test set: Average loss: 0.1165, Accuracy: 9817/10000 (98%)

